In [10]:
%pip install folium requests



You should consider upgrading via the '/Users/I528946/.pyenv/versions/AiImageDetection/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import subprocess
import re
import requests
import folium
import time

# === CONFIGURATION ===
MAX_HOPS = "10"
TARGET = "sreenidhi.edu.in"
API_URL = "http://ip-api.com/json/"

# === STEP 1: Run traceroute ===
def run_traceroute(domain):
    print(f"Running traceroute to {domain} (max {MAX_HOPS} hops)...")
    result = subprocess.run(["traceroute", "-m", MAX_HOPS, domain], stdout=subprocess.PIPE, timeout=60)
    return result.stdout.decode()

# === STEP 2: Extract IPs ===
def extract_ips(output):
    ip_pattern = re.compile(r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b")
    ips = ip_pattern.findall(output)
    public_ips = [ip for ip in ips if not ip.startswith(("10.", "192.168.", "172."))]
    return list(dict.fromkeys(public_ips))  # Remove duplicates

# === STEP 3: Get Geo info ===
def get_geolocation(ip):
    try:
        res = requests.get(f"{API_URL}{ip}", timeout=3).json()
        if res['status'] == 'success':
            return {
                'ip': ip,
                'lat': res['lat'],
                'lon': res['lon'],
                'city': res['city'],
                'country': res['country']
            }
    except requests.RequestException:
        pass
    return None

# === STEP 4: Plot map ===
def plot_map(locations, target):
    valid = [loc for loc in locations if loc]
    if not valid:
        print("No valid IPs to plot.")
        return
    m = folium.Map(location=[valid[0]['lat'], valid[0]['lon']], zoom_start=4)
    for i, loc in enumerate(valid):
        folium.Marker(
            [loc['lat'], loc['lon']],
            popup=f"{i+1}. {loc['ip']} ({loc['city']}, {loc['country']})"
        ).add_to(m)
    folium.PolyLine([(loc['lat'], loc['lon']) for loc in valid], color="blue").add_to(m)
    filename = f"traceMap_{target}.html"
    m.save(filename)
    print(f"✅ Map saved as: {filename}")

# === MAIN ===
if __name__ == "__main__":
    try:
        out = run_traceroute(TARGET)
        ips = extract_ips(out)
        print(f"Found public IPs: {ips}")

        # Geolocate with small delay to avoid rate-limiting
        locations = []
        for ip in ips:
            loc = get_geolocation(ip)
            if loc:
                print(f"✔ {ip} → {loc['city']}, {loc['country']}")
                locations.append(loc)
            else:
                print(f"✖ No data for {ip}")
            time.sleep(0.5)  # Avoid API rate limit

        plot_map(locations, TARGET.replace(".", "_"))

    except subprocess.TimeoutExpired:
        print("⏱️ Traceroute timed out.")
    except Exception as e:
        print(f"❌ Error: {e}")


Running traceroute to sreenidhi.edu.in (max 10 hops)...


traceroute to sreenidhi.edu.in (119.18.54.196), 10 hops max, 40 byte packets


Found public IPs: ['103.109.144.1', '203.78.211.101', '202.75.207.217', '140.222.7.71', '140.222.7.73', '115.113.165.233']
✔ 103.109.144.1 → Bengaluru, India
✔ 203.78.211.101 → New Delhi, India
✔ 202.75.207.217 → Mumbai, India
✔ 140.222.7.71 → Harrison, United States
✔ 140.222.7.73 → Harrison, United States
✔ 115.113.165.233 → Mumbai, India
✅ Map saved as: traceMap_sreenidhi_edu_in.html
